# Assignment 1 Verification Script
This script will package your assignment, put it in a submissions folder, generate a checksum and notify the submissions server that you have a submission you wish to submit. When it gets around to it, the submission server will then pull your submission and let you know if it is in the right form. As this system is experimental this year and due to university account-related logistical issues, you will have to upload your submission to Canvas as well.

When you submit, the script will check some basic things locally before submitting your assignment. This verification will occur once more in the submission environment in an attempt to ensure you haven't forgotten anything in your submission.

## What you need to do
1. Set the folder you want to submit:


In [5]:
import datetime
submit_folder = "/home/jovyan/assignments/example"
print("[", datetime.datetime.now(), "]", "Set submit folder to", submit_folder)

[ 2020-04-28 23:48:27.106132 ] Set submit folder to /home/jovyan/assignments/example


2. Run the submission setup script.

In [7]:
import os
import subprocess
import zipfile
import submitutils
import time

# The folder where data is persisted
BASE_FOLDER = "/home/jovyan"

# The folder where submissions should be placed, should be relative to mount point. This can be safely changed.
SUBMISSIONS_FOLDER = "submissions"

# The assignment being submitted to
ASSIGNMENT_NAME = "ass1"

# The verify script to test the submission should pass
VERIFY_SCRIPT_LOCATION = "/home/jovyan/assignments/verify.sh"

# The location to store all messages recieved on the queue.
MONITOR_FILE = "/home/jovyan/assignments/submissions.log"

print("[", datetime.datetime.now(), "]", "Using submission folder", submit_folder, "for submission.")

# Find latest file in file list.
list_of_files = submitutils.file_list(submit_folder)
#print(list_of_files)
list_of_files.sort(key=lambda x: x['modified_s'], reverse=True)
list_of_filenames = [(file['filename'], file['timestamp'], file['raw_filesize']) for file in list_of_files]
last_modified_file = list_of_files[0]

submission_zipfile_base = last_modified_file['timestampstring'] + '_' + ASSIGNMENT_NAME
submission_zipfile_name = submission_zipfile_base + ".zip"
print("[", datetime.datetime.now(), "]", "Storing files in zip file", submission_zipfile_name, "for submission.")

print("[", datetime.datetime.now(), "]", "Submission contents:")
print("\tFilename\tModified")
BOLD_START = "\033[1m"
END_FORMATTING = "\033[0m"
for file, modified, size in list_of_filenames:
    if modified == list_of_filenames[0][1]:
        print(BOLD_START, end="")
    print("\t" + file + "\t" + modified + "\t" + submitutils.get_friendly_size(size))
    if modified == list_of_filenames[0][1]:
        print(END_FORMATTING, end="")

base_relative_folder = SUBMISSIONS_FOLDER + "/" + ASSIGNMENT_NAME
submission_zipfile_folder = BASE_FOLDER + "/" + base_relative_folder
submission_zipfile_location = submission_zipfile_folder + "/" + submission_zipfile_name
print("[", datetime.datetime.now(), "]", "Checking if",  submission_zipfile_location, "already exists")
if os.path.exists(submission_zipfile_location):
    print("[", datetime.datetime.now(), "]", submission_zipfile_location, "already exists, delete the file if you intend to resubmit this submission.")
else:
    # Ensure the folder exists
    os.popen('mkdir -p {}'.format(submission_zipfile_folder)).read()
    with zipfile.ZipFile(submission_zipfile_location, mode="w", compression=zipfile.ZIP_LZMA) as zf:
        for file in list_of_files:
            zf.write(file['path'], arcname=file['relative_path'])
    zip_hash = submitutils.get_file_hash(submission_zipfile_location)
    print("[", datetime.datetime.now(), "]", "Submission hash:", zip_hash)
    zip_hash_location = submission_zipfile_folder + "/" + submission_zipfile_base + ".sha1"
    with open(zip_hash_location, "w") as f:
        f.write(zip_hash)
    
    # Verify submission locally
    print("")
    print("[", datetime.datetime.now(), "]", "Verifying submission locally")
    SUBMISSION_FOLDER = submit_folder
    VERIFICATION_OUTPUT_FOLDER = "/home/jovyan/assignments/output"
    os.popen("mkdir -p {}".format(VERIFICATION_OUTPUT_FOLDER)).read()
    VERIFICATION_OUTPUT_FILE = "verify_out.txt"
    USERNAME = "N/A"
    VERIFICATION_EXTRA_DATA_FOLDER = "N/A"
    TIMEOUT = 124
    verify_command = ["/bin/bash", VERIFY_SCRIPT_LOCATION, SUBMISSION_FOLDER, VERIFICATION_OUTPUT_FOLDER, USERNAME, VERIFICATION_EXTRA_DATA_FOLDER]
    verify_success = False
    try:
        subprocess.check_call(verify_command)
        if os.path.exists(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE):
            with open(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE, "r") as f:
                print(f.read())
        verify_success = True
    except subprocess.CalledProcessError as e:
        
        print("[", datetime.datetime.now(), "]", "Verification failed:")
        print(e.cmd)
        if os.path.exists(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE):
            with open(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE, "r") as f:
                print(f.read())
        if e.returncode == TIMEOUT:
            print("Error Code:", e.returncode, "Process ran out of time")
        else:
            print("Error Code:", e.returncode, "Process ran out of time")
    if verify_success:
        # Submit to Assignment queue
        print("")
        
        # Submit mode is "submit" when user disk control is used as authorization.
        # Submit mode is "psubmit" when command signing is used as authorization.
        submit_mode = "submit"
        username = submitutils.get_username()
        print("[", datetime.datetime.now(), "]", "Submitting assignment to queue")
        print("submission type: ", submit_mode)
        print("username: ", username)
        print("assignment: ", ASSIGNMENT_NAME)
        print("checksum: ", zip_hash)
        final_submit_path = base_relative_folder + "/" + submission_zipfile_name
        print("path: ", final_submit_path)
        print("submission size: ", submitutils.get_friendly_size(os.path.getsize(submission_zipfile_location)))
        if not submitutils.submit_to_queue(submit_mode="submit", username=username, assignment=ASSIGNMENT_NAME, zip_hash=zip_hash, submit_path=final_submit_path):
            raise Exception("Missing required field")
        print("[", datetime.datetime.now(), "]", "Beginning listen for queue, wait until verification completes", username)
        pass_result = submitutils.monitor_queue(queue=username, complete_stage="verify", monitor_file=MONITOR_FILE, submission_name=final_submit_path) is "pass"
        print("[", datetime.datetime.now(), "]", "Verification completed, result:", pass_result)
        if not pass_result:
            print("[", datetime.datetime.now(), "]", "Verification failed, please wait for all processing to finish and read the logs.")
        else:
            print("[", datetime.datetime.now(), "]", "Please save your submitted zip file and message log")
            submitutils.monitor_queue(queue=username, complete_stage="test", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
            print("[", datetime.datetime.now(), "]", "Completed testing")
            submitutils.monitor_queue(queue=username, complete_stage="results", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
            print("[", datetime.datetime.now(), "]", "Finished evaluating test results")
            submitutils.monitor_queue(queue=username, complete_stage="finalise", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
            print("[", datetime.datetime.now(), "]", "Testing finished, results ready")
            
            communication_file = submission_zipfile_folder + "/" + submission_zipfile_base + ".out"
            if os.path.exists(communication_file):
                with open(communication_file, "r") as f:
                    print(f.read())
            else:
                print("Failed to read communication file.")
        
        
    

[ 2020-04-28 23:49:04.368280 ] Using submission folder /home/jovyan/assignments/example for submission.
[ 2020-04-28 23:49:04.448056 ] Storing files in zip file 20200428232501_ass1.zip for submission.
[ 2020-04-28 23:49:04.449064 ] Submission contents:
	Filename	Modified
	Assignment_Solution.py	Tue Apr 28 23:25:01 2020	10.1KiB
	cheeky.py.hide	Tue Apr 28 23:00:37 2020	53 bytes
	out1.txt	Tue Apr 14 23:38:57 2020	1.4KiB
	out2.txt	Tue Apr 14 23:38:57 2020	19 bytes
	task5.png	Tue Apr 14 23:38:49 2020	4.7KiB
	task3.csv	Tue Apr 14 23:34:25 2020	77 bytes
	Untitled.ipynb	Tue Apr 14 19:24:17 2020	29.1KiB
	task4.png	Tue Apr 14 14:15:15 2020	9.8KiB
	task2.csv	Tue Apr 14 10:33:28 2020	400 bytes
	untitled.txt	Tue Apr 14 10:25:45 2020	0 bytes
	task1.csv	Tue Apr 14 10:06:37 2020	351 bytes
	example.py.hide	Sun Mar 29 14:14:33 2020	31 bytes
	example2.py.hide	Sun Mar 29 14:12:06 2020	28 bytes
[ 2020-04-28 23:49:04.452305 ] Checking if /home/jovyan/submissions/ass1/20200428232501_ass1.zip already exists
[

KeyboardInterrupt: 